In [2]:
!pip install easyocr pdf2image python-docx pandas openpyxl pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42

In [3]:
import easyocr
import pandas as pd
from PIL import Image, ImageDraw
from docx import Document
import os
import numpy as np

# Initialize EasyOCR
reader = easyocr.Reader(['en'])

def docx_to_image(docx_path):
    doc = Document(docx_path)
    text = '\n'.join([para.text for para in doc.paragraphs])
    img = Image.new('RGB', (1200, 1600), color='white')
    draw = ImageDraw.Draw(img)
    draw.text((20, 20), text, fill='black')
    return img

def extract_text_easyocr(image):
    # Convert PIL Image to numpy array
    image_np = np.array(image)
    result = reader.readtext(image_np, detail=0)
    return '\n'.join(result)

def parse_text_to_rows(text, attributes):
    rows = []
    current_row = {}
    for line in text.splitlines():
        line = line.strip()
        for attr in attributes:
            if line.lower().startswith(attr.lower()):
                value = line.split(":", 1)[-1].strip() if ':' in line else line[len(attr):].strip()
                current_row[attr] = value
        if len(current_row) == len(attributes):
            rows.append(current_row)
            current_row = {}
    return pd.DataFrame(rows)

def convert_to_excel(input_path, output_path, attributes):
    ext = os.path.splitext(input_path)[-1].lower()

    if ext in [".jpg", ".jpeg", ".png"]:
        image = Image.open(input_path)
    elif ext == ".docx":
        image = docx_to_image(input_path)
    else:
        raise ValueError("Only .jpg, .png, .docx supported without external binaries.")

    text = extract_text_easyocr(image)
    df = parse_text_to_rows(text, attributes)
    df.to_excel(output_path, index=False)
    print(f"✅ Saved Excel to: {output_path}")
    return df

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
input_file = "/content/Swetha_Yanni_IAM.docx"  # or "sample.jpg"
output_excel = "output.xlsx"
attributes = ["Name", "Email", "Phone", "Address"]

df = convert_to_excel(input_file, output_excel, attributes)
df.head()

✅ Saved Excel to: output.xlsx


,Name,Phone,Email,Address
0,Swetha Yanni,. 836-287-7009,Sivethayprahallad,@Linkedln: https linkedincom iny-sietha
